In [9]:
import os
import pandas as pd
import mysql.connector

# Database connection details
DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "uct04bgs101120",
    "database": "agric_cameroon"
}

# Ensure output directory exists
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

# Establish a connection to the database
conn = mysql.connector.connect(**DB_CONFIG)

# Queries for the features (Drought_Index query removed)
queries = {
    "Avg_Annual_Production": """
    SELECT subquery.Region_Name, 
           subquery.Year,
           SUM(subquery.Avg_Annual_Production) AS Total_Annual_Production
    FROM (
        SELECT region_dim.Region_Name, 
               time_dim.Year,
               AVG(production_fact.Quantity) AS Avg_Annual_Production
        FROM production_fact
        JOIN time_dim ON production_fact.Time_ID = time_dim.Time_ID
        JOIN region_dim ON production_fact.Region_ID = region_dim.Region_ID
        WHERE time_dim.Year BETWEEN 2019 AND 2024
        GROUP BY region_dim.Region_Name, time_dim.Year
    ) AS subquery
    GROUP BY subquery.Region_Name, subquery.Year;
    """,

    "Production_Variability": """
        SELECT region_dim.Region_Name, 
               STDDEV(production_fact.Quantity) AS Production_Variability
        FROM production_fact
        JOIN time_dim ON production_fact.Time_ID = time_dim.Time_ID
        JOIN region_dim ON production_fact.Region_ID = region_dim.Region_ID
        WHERE time_dim.Year BETWEEN 2019 AND 2024
        GROUP BY region_dim.Region_Name;
    """,

    "Avg_Temperature": """
        SELECT region_dim.Region_Name, 
               AVG(climate_dim.Avg_Temperature) AS Avg_Temperature
        FROM region_dim
        JOIN climate_dim ON region_dim.Climate_ID = climate_dim.Climate_ID
        GROUP BY region_dim.Region_Name;
    """,

    "Annual_Rainfall": """
        SELECT region_dim.Region_Name, 
               AVG(climate_dim.Annual_Rainfall) AS Annual_Rainfall
        FROM region_dim
        JOIN climate_dim ON region_dim.Climate_ID = climate_dim.Climate_ID
        GROUP BY region_dim.Region_Name;
    """,

    "Moisture_Index": """
        SELECT region_dim.Region_Name, 
               AVG(climate_dim.Annual_Rainfall - (climate_dim.Avg_Temperature * 10)) AS Moisture_Index
        FROM region_dim
        JOIN climate_dim ON region_dim.Climate_ID = climate_dim.Climate_ID
        GROUP BY region_dim.Region_Name;
    """
}

# Execute each query and save results to individual CSV files
for feature, query in queries.items():
    try:
        df = pd.read_sql_query(query, conn)
        output_file = os.path.join(output_dir, f"{feature}.csv")
        df.to_csv(output_file, index=False)
        print(f"Saved {feature} to '{output_file}'")
    except Exception as e:
        print(f"Error executing query for {feature}: {e}")

# Close the database connection
conn.close()

C:\Users\huawei\AppData\Local\Temp\ipykernel_13372\930035495.py:77: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


Saved Avg_Annual_Production to './output\Avg_Annual_Production.csv'
Saved Production_Variability to './output\Production_Variability.csv'
Saved Avg_Temperature to './output\Avg_Temperature.csv'
Saved Annual_Rainfall to './output\Annual_Rainfall.csv'
Saved Moisture_Index to './output\Moisture_Index.csv'
